In [ ]:
import rqdatac
import pandas as pd
import statsmodels.api as sm
from linearmodels.panel import FamaMacBeth
import csv
import numpy as np
import import_ipynb
import a_general_factor_test
from scipy.cluster import hierarchy # 导入我们用于层次聚类的新工具
from sklearn.preprocessing import StandardScaler # 导入用于数据标准化的新工具


rqdatac.init()
start_date = pd.to_datetime('2018-01-01')
end_date = pd.to_datetime('2020-12-31')
instrument_type = 'Convertible'
factor_name = '第二产业增加值占GDP比重(现价)'
remaining_time_to_mature ='短期 (1-3年)' #labels = ['短期 (1-3年)', '中期 (3-5年)', '长期 (5+年)']
rolling_window = 90
folder_name = f'econ_factor_{remaining_time_to_mature}'


In [9]:
def econ_get_t_value_series(all_instruments,dependent,start_date,end_date,instrument_type,factor_name,remaining_time_to_mature,rolling_window,folder_name):

    try:
        independent = a_general_factor_test.econ_get_factor_value_forward_fill(factor_name,start_date,end_date)
        time_regression_df = a_general_factor_test.prepare_time_regression_df(dependent,independent)
        rolling_betas = a_general_factor_test.roll_data_and_regress(time_regression_df, rolling_window)
        #cb only
        rolling_betas_with_timegroup_tag = a_general_factor_test.add_maturity_group_tag(rolling_betas,all_instruments)
        rolling_betas = a_general_factor_test.select_wanted_rolling_data(rolling_betas_with_timegroup_tag,remaining_time_to_mature)
        result = a_general_factor_test.fama_macbeth_manual_for_t_value(rolling_betas,dependent)
        a_general_factor_test.dump_t_series(result,folder_name,factor_name)
    except Exception as e:
        print(e)
'''start_date = pd.to_datetime('2018-01-01')
end_date = pd.to_datetime('2020-12-31')
instrument_type = 'Convertible'
factor_name = '第二产业增加值占GDP比重(现价)'
remaining_time_to_mature ='短期 (1-3年)'
rolling_window = 90'''

"start_date = pd.to_datetime('2018-01-01')\nend_date = pd.to_datetime('2020-12-31')\ninstrument_type = 'Convertible'\nfactor_name = '第二产业增加值占GDP比重(现价)'\nremaining_time_to_mature ='短期 (1-3年)'\nrolling_window = 90"

In [10]:
import pandas as pd
import re # 导入正则表达式库，用于更精确的文本匹配

def extract_factor_names_from_results(file_path: str) -> list[str]:
    """
    从一个特定格式的结果文件中读取并提取所有因子的名称。

    这个函数专门用于解析如下格式的文本行：
    'factor_name:  [因子名称]    p_value: [p值]  t_value:[t值]'

    Args:
        file_path (str): 结果文件的完整路径。

    Returns:
        list[str]: 一个包含所有提取出的因子名称的Python列表。
    """
    factor_names = [] # 1. 初始化一个空列表，用来存放结果
    
    print(f"正在从文件 {file_path} 中读取因子列表...")
    
    try:
        # 2. 使用 'with open' 安全地打开文件
        # 'r' 代表读取模式, 'encoding='utf-8'' 确保能正确处理中文字符
        with open(file_path, 'r', encoding='utf-8') as f:
            
            # 3. 逐行遍历文件中的每一行文本
            for line in f:
                # 4. 检查这一行是否是我们需要的格式
                if "factor_name:" in line and "p_value:" in line:
                    
                    # 5. 【核心】使用正则表达式来精确提取
                    # re.search() 会在字符串中寻找匹配的模式
                    # 模式 r'factor_name:\s*(.*?)\s*p_value:' 的意思是：
                    #   - 'factor_name:' : 匹配 "factor_name:"
                    #   - '\s*' : 匹配任意数量的空格
                    #   - '(.*?)' : 这是一个捕获组，匹配中间的所有字符（非贪婪模式），这就是我们想要的因子名！
                    #   - '\s*' : 再次匹配任意数量的空格
                    #   - 'p_value:' : 匹配 "p_value:"
                    match = re.search(r'factor_name:\s*(.*?)\s*p_value:', line)
                    
                    if match:
                        # 6. 如果成功匹配，提取捕获组的内容
                        factor_name = match.group(1).strip() # .strip() 清理一下可能残留的空格
                        factor_names.append(factor_name)
                        
    except FileNotFoundError:
        print(f"错误：找不到文件 '{file_path}'。请检查路径是否正确。")
        return [] # 如果文件不存在，返回一个空列表
        
    except Exception as e:
        print(f"读取文件时发生未知错误: {e}")
        return []

    print(f"读取完成，共找到 {len(factor_names)} 个因子。")
    return factor_names
factor_name_list = extract_factor_names_from_results('/Users/zhanghongyi/Desktop/25_Summer/Quant_Finance/ricequant_strategies/Factor_test/econ_短期 (1-3年)_factor/Result_file_短期 (1-3年).csv')

正在从文件 /Users/zhanghongyi/Desktop/25_Summer/Quant_Finance/ricequant_strategies/Factor_test/econ_短期 (1-3年)_factor/Result_file_短期 (1-3年).csv 中读取因子列表...
读取完成，共找到 129 个因子。


In [11]:
remaining_time_to_mature = ['短期 (1-3年)']
all_instruments = a_general_factor_test.get_bonds_poll(start_date, end_date, instrument_type)
dependent = a_general_factor_test.get_return_rate(all_instruments,start_date,end_date)
for mature_remain in remaining_time_to_mature:
    folder_name = f'd_{mature_remain}_tvalue_collection'

    for factor_name in factor_name_list:
        cleaned_name = str(factor_name).strip().strip("'").strip('"')
        econ_get_t_value_series(
                                all_instruments = all_instruments,
                                start_date=start_date,
                                dependent=dependent
                                                             ,end_date= end_date
                                                             ,instrument_type=instrument_type
                                                             ,factor_name = cleaned_name
                                                             ,remaining_time_to_mature = mature_remain
                                                             ,rolling_window =rolling_window,
                                                             folder_name = folder_name)

testmodel2
可转债数量: 449
testing: 工业品出厂价格指数PPI_(上年=100)_当月
✅ 因子 工业品出厂价格指数PPI_(上年=100)_当月 的t值序列已保存至 d_短期 (1-3年)_tvalue_collection/index_工业品出厂价格指数PPI_(上年=100)_当月.csv
testing: 居民粮食消费价格指数CPI_(上年=100)_累计
✅ 因子 居民粮食消费价格指数CPI_(上年=100)_累计 的t值序列已保存至 d_短期 (1-3年)_tvalue_collection/index_居民粮食消费价格指数CPI_(上年=100)_累计.csv
testing: 工业品购进价格指数PPIRM(上年=100)_累计同比


KeyboardInterrupt: 